# Feature-Extraction

Extraction of features of each of the frames of the video.
Code by:

    Krishna Srikar Durbha - EE18BTECH11014
    V. L. Narasimha Reddy - EE18BTECH11046

## Imports

### Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import imageio
import cv2
import scipy
import os
import random
from tqdm import tqdm

# Tensorflow Libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, GRU, Dense, Input, LSTM, GRU, Dropout, Reshape

### Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Video-Classification

## Feature Extraction Model

Model to extract features from images or in this case video frames.

In [ ]:
# Feature Extraction Model
FeatureExtractor = tf.keras.applications.inception_v3.InceptionV3(
    include_top = False,
    pooling = "max"
)
FeatureExtractor.trainable = False

# Saving Feature Extraction Model
FeatureExtractor.save("Models/Feature-Extraction_Model.h5")

# Input Preprocessing Function
Preprocess_Input = tf.keras.applications.inception_v3.preprocess_input

## Extracting and Saving Features

#### Extracting Frames from Video

In [ ]:
def ExtractFrames(Path, resize=(128, 128)):
    cap = cv2.VideoCapture(Path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
    finally:
        cap.release()
    return np.array(frames)

#### Dataset

In [ ]:
def SaveFeatures(Videos_Path, Features_Path):
    Classes = os.listdir(Videos_Path)

    for i in tqdm(range(len(Classes))):
        # Label
        label = Classes[i]

        # Creating a Directory
        os.mkdir(Features_Path + "/" + label)

        for f in os.listdir(Videos_Path + "/" + label):
            # Extracting Frames
            path = Videos_Path + "/" + label + "/" + f
            try:
                Frames = ExtractFrames(path)
            except:
                FrameData = ExtractFrames(path.decode("utf-8"))

            # Features
            Features = FeatureExtractor.predict(Preprocess_Input(Frames))

            # Saving Features
            np.save(Features_Path + "/" + label + "/" + f[:-4], Features)

#### Saving Features

In [ ]:
Videos_Path = "/Datasets/Videos"
Features_Path = "/Datasets/Features"

SaveFeatures(Videos_Path, Features_Path)